In [2]:
"""
清洗问卷数据，主要处理出发地点和到达地点问题，并且添加距离
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', font='simhei')

In [303]:
path = r'D:\Zhejiang University\Graduate Project\Data\Data\Cleaned\trip_questionaire.csv'
person_path = r'D:\Zhejiang University\Graduate Project\Data\Data\person.csv'
data = pd.read_csv(path).iloc[:, 1:]
person = pd.read_csv(person_path)
data.head()

,用户ID,星期,日期,Trip_ID,出行目的,第几次出行,出发地点,到达地点,出发时间,达到时间,...,性别,年龄,宿舍区,年级,专业大类,自行车保有,电动车保有,汽车保有,校内主要出行方式,校外主要出行方式
0,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0001,4,1,翠柏二舍,安中大楼B,7:40,8:03,...,1,23,4,6,土木工程,2,2,2,1-2,1-2-7-8-9
1,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0002,3,2,安中大楼B,风味食堂,11:15,11:25,...,1,23,4,6,土木工程,2,2,2,1-2,1-2-7-8-9
2,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0003,6,3,风味食堂,翠柏二舍,11:40,11:46,...,1,23,4,6,土木工程,2,2,2,1-2,1-2-7-8-9
3,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0004,4,4,翠柏二舍,安中大楼B,14:15,14:31,...,1,23,4,6,土木工程,2,2,2,1-2,1-2-7-8-9
4,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0005,2,5,安中大楼B,西2,14:37,14:42,...,1,23,4,6,土木工程,2,2,2,1-2,1-2-7-8-9


# 观察地点
- 初步整理

In [304]:
start_pos = data['出发地点'].unique()
print(start_pos)
print('-----------------------------------------------------------------------------------------------------')
end_pos = data['到达地点'].unique()
print(end_pos)

['翠柏二舍' '安中大楼B' '风味食堂' '西2' '安中B' '休闲食堂' '东1' '校医院' '健身房' '青溪四舍' '东6教学楼'
 '堕落街' '西2楼食堂' '南华园' '剑桥公社' '白沙3舍' '农生环A' '白沙3' '浦东机场' '白3' '农A' '白沙二舍'
 '安中大楼' '食堂' '白沙三舍' '蒙民伟楼' '白沙一舍' '大食堂' '安中' '翠柏一舍' '休闲餐厅' '安中A' '临湖餐厅'
 '西区2楼餐厅' '建工实验大厅' '翠柏' '实验大厅' '蓝田4舍' '东六教学楼' '东四教学楼' '西2食堂' '紫云3' '西1'
 '西2教学楼' '图书馆' '小剧场' '格外' '西1教学楼' '西田径场' '东田径场' '西4教学楼' '西4' '风味餐厅' '蓝田四舍'
 '东6' '玉泉图书馆' '翠柏四舍' '董' '西操' '码头' '临湖教超' '东2' '化学实验中心' '蓝田' '东操' '紫云'
 '蓝田4' '蓝4' '丹阳5' '休闲' '丹5' '风味' '麦香食堂' '临湖' '西区食堂' '建工试验大厅' '西四教学楼'
 '东区食堂' '东一教学楼' '城西银泰D座' '白沙四舍' '安中A座' '二楼食堂' '风雨操场' '丹阳' '文三路' '白沙二'
 '临湖食堂' '白沙' '白沙2' '小卖部' '东2食堂' '启真便利店' '白沙启真便利店' '试验大厅' '西面2楼食堂' '紫金港东操场'
 '白2' '紫金港公交站' '东操场' '建工大厅' '云峰' '农生环' '机房' '东4' '麦香餐厅' '生物实验中心' '翠4'
 '小河直街' 'C4' '月牙楼' '东1B' '翠2' '网球场' '东大门' '东1教学楼' '金工中心' '西一四']
-----------------------------------------------------------------------------------------------------
['安中大楼B' '风味食堂' '翠柏二舍' '西2' '安中B' '休闲食堂' '东1' '校医院' '健身房' '东6教学楼' '堕落街'
 '青溪四舍' '西2楼食堂' '西2食堂' '南华园' '剑桥公社' '农生环A' 

## 整理地点
规则如下：
- 宿舍名+数字，如无数字，则直接宿舍名
- 排除校外地点
- 排除未知地点
- 缩减食堂范围
- 安中大楼A或安中大楼B或安中大楼
- 东/西教学楼+数字或东/西教学楼

In [305]:
# 宿舍
for pos in start_pos:
    if '青' in pos or '翠' in pos or '碧' in pos or '紫' in pos or '白' in pos or '丹' in pos or '蓝' in pos:
        print(pos)

翠柏二舍
青溪四舍
白沙3舍
白沙3
白3
白沙二舍
白沙三舍
白沙一舍
翠柏一舍
翠柏
蓝田4舍
紫云3
蓝田四舍
翠柏四舍
蓝田
紫云
蓝田4
蓝4
丹阳5
丹5
白沙四舍
丹阳
白沙二
白沙
白沙2
白沙启真便利店
紫金港东操场
白2
紫金港公交站
翠4
翠2


---
- 除 白沙启真便利店，紫金港东操场外， 紫金港公交站外都可修正

In [306]:
# 食堂
for pos in start_pos:
    if '临湖' in pos or '麦香' in pos or '食堂' in pos:
        print(pos)

风味食堂
休闲食堂
西2楼食堂
食堂
大食堂
临湖餐厅
西2食堂
临湖教超
麦香食堂
临湖
西区食堂
东区食堂
二楼食堂
临湖食堂
东2食堂
西面2楼食堂
麦香餐厅


In [307]:
# 教学楼
for pos in start_pos:
    if '东' in pos or '西' in pos:
        print(pos)

西2
东1
东6教学楼
西2楼食堂
浦东机场
西区2楼餐厅
东六教学楼
东四教学楼
西2食堂
西1
西2教学楼
西1教学楼
西田径场
东田径场
西4教学楼
西4
东6
西操
东2
东操
西区食堂
西四教学楼
东区食堂
东一教学楼
城西银泰D座
东2食堂
西面2楼食堂
紫金港东操场
东操场
东4
东1B
东大门
东1教学楼
西一四


In [308]:
def correct_dormitory(name, pos):
    if '1' in pos or '一' in pos:
        new_pos = name + '1'
    elif '2' in pos or '二' in pos:
        new_pos = name + '2'
    elif '3' in pos or '三' in pos:
        new_pos = name + '3'
    elif '4' in pos or '四' in pos:
        new_pos = name + '4'
    else:
        new_pos = name
    return new_pos

In [309]:
# 初步修正宿舍，食堂，教学楼，操场
new_start_pos = []
for i, row in data.iterrows():
    s_pos = row['出发地点']
    # 宿舍
    # 除 白沙启真便利店，紫金港东操场外， 紫金港公交站外都可修正
    if '蓝' in s_pos:
        n_pos = correct_dormitory('蓝田', s_pos)
    elif '碧' in s_pos:
        n_pos = correct_dormitory('碧峰', s_pos)
    elif '紫云' in s_pos:
        n_pos = correct_dormitory('紫云', s_pos)
    elif '丹' in s_pos:
        n_pos = correct_dormitory('丹阳', s_pos)
    elif '青' in s_pos:
        n_pos = correct_dormitory('青溪', s_pos)
    elif '翠' in s_pos:
        n_pos = correct_dormitory('翠柏', s_pos)
    elif '白' in s_pos:
        if '启真' not in s_pos:
            n_pos = correct_dormitory('白沙', s_pos)
        else:
            n_pos = s_pos
            
    # 食堂
    # 临湖教超除外
    elif '临湖' in s_pos and '教超' not in s_pos:
        n_pos = '临湖食堂'
    elif '麦香' in s_pos:
        n_pos = '麦香餐厅'
    elif '食堂' in s_pos or '休闲' in s_pos or '风味' in s_pos:
        n_pos = '生活区大食堂'
    
    # 教室
    elif '东' in s_pos and '浦东' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '东田径场'
        else:
            n_pos = correct_dormitory('东区教学楼', s_pos)
    elif '西' in s_pos and '银泰' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '西田径场'
        else:
            n_pos = correct_dormitory('西区教学楼', s_pos)
    
    # 安中大楼
    elif '安中' in s_pos:
        if 'A' in s_pos or 'a' in s_pos:
            n_pos = '安中大楼A'
        elif 'B' in s_pos or 'b' in s_pos:
            n_pos = '安中大楼B'
        else:
            n_pos = '安中大楼'
    elif '建工' in s_pos or '大厅' in s_pos:
        n_pos = '建工试验大厅'
    elif '小卖部' in s_pos or '启真便利店' in s_pos:
        n_pos = '白沙启真便利店'
    elif '农' in s_pos:
        n_pos = '农生环'
    else:
        n_pos = s_pos
    new_start_pos.append(n_pos)
    print(s_pos, '-->', n_pos)

翠柏二舍 --> 翠柏2
安中大楼B --> 安中大楼B
风味食堂 --> 生活区大食堂
翠柏二舍 --> 翠柏2
安中大楼B --> 安中大楼B
西2 --> 西区教学楼2
安中大楼B --> 安中大楼B
翠柏二舍 --> 翠柏2
安中B --> 安中大楼B
风味食堂 --> 生活区大食堂
翠柏二舍 --> 翠柏2
安中B --> 安中大楼B
休闲食堂 --> 生活区大食堂
安中B --> 安中大楼B
东1 --> 东区教学楼1
安中B --> 安中大楼B
翠柏二舍 --> 翠柏2
风味食堂 --> 生活区大食堂
校医院 --> 校医院
安中B --> 安中大楼B
西2 --> 西区教学楼2
风味食堂 --> 生活区大食堂
安中B --> 安中大楼B
翠柏二舍 --> 翠柏2
健身房 --> 健身房
风味食堂 --> 生活区大食堂
青溪四舍 --> 青溪4
东6教学楼 --> 东区教学楼
堕落街 --> 堕落街
青溪四舍 --> 青溪4
西2楼食堂 --> 生活区大食堂
青溪四舍 --> 青溪4
西2楼食堂 --> 生活区大食堂
青溪四舍 --> 青溪4
风味食堂 --> 生活区大食堂
东6教学楼 --> 东区教学楼
堕落街 --> 堕落街
东6教学楼 --> 东区教学楼
青溪四舍 --> 青溪4
南华园 --> 南华园
剑桥公社 --> 剑桥公社
白沙3舍 --> 白沙3
农生环A --> 农生环
白沙3 --> 白沙3
浦东机场 --> 浦东机场
白3 --> 白沙3
农A --> 农生环
白沙3 --> 白沙3
农A --> 农生环
白沙二舍 --> 白沙2
安中大楼 --> 安中大楼
食堂 --> 生活区大食堂
白沙三舍 --> 白沙3
安中大楼 --> 安中大楼
食堂 --> 生活区大食堂
安中大楼 --> 安中大楼
白沙二舍 --> 白沙2
食堂 --> 生活区大食堂
白沙二舍 --> 白沙2
安中大楼 --> 安中大楼
白沙二舍 --> 白沙2
安中大楼 --> 安中大楼
白沙二舍 --> 白沙2
安中大楼 --> 安中大楼
食堂 --> 生活区大食堂
安中大楼 --> 安中大楼
食堂 --> 生活区大食堂
白沙二舍 --> 白沙2
蒙民伟楼 --> 蒙民伟楼
白沙一舍 --> 白沙1
大食堂 --> 生活区大食堂
安中 --> 安中大楼
食堂 --

In [310]:
data['修正后出发地点'] = new_start_pos
print(data['修正后出发地点'].unique())

['翠柏2' '安中大楼B' '生活区大食堂' '西区教学楼2' '东区教学楼1' '校医院' '健身房' '青溪4' '东区教学楼' '堕落街'
 '南华园' '剑桥公社' '白沙3' '农生环' '浦东机场' '白沙2' '安中大楼' '蒙民伟楼' '白沙1' '翠柏1' '安中大楼A'
 '临湖食堂' '建工试验大厅' '翠柏' '蓝田4' '东区教学楼4' '紫云3' '西区教学楼1' '图书馆' '小剧场' '格外'
 '西区教学楼' '西区教学楼4' '玉泉图书馆' '翠柏4' '董' '西田径场' '码头' '临湖教超' '东区教学楼2' '化学实验中心'
 '蓝田' '东田径场' '紫云' '丹阳' '麦香餐厅' '城西银泰D座' '白沙4' '风雨操场' '文三路' '白沙' '白沙启真便利店'
 '紫金港公交站' '云峰' '机房' '生物实验中心' '小河直街' 'C4' '月牙楼' '网球场' '金工中心']


In [311]:
# 修正达到地点
new_end_pos = []
for i, row in data.iterrows():
    s_pos = row['到达地点']
    # 宿舍
    # 除 白沙启真便利店，紫金港东操场外， 紫金港公交站外都可修正
    if '蓝' in s_pos:
        n_pos = correct_dormitory('蓝田', s_pos)
    elif '碧' in s_pos:
        n_pos = correct_dormitory('碧峰', s_pos)
    elif '紫云' in s_pos:
        n_pos = correct_dormitory('紫云', s_pos)
    elif '丹' in s_pos:
        n_pos = correct_dormitory('丹阳', s_pos)
    elif '青' in s_pos:
        n_pos = correct_dormitory('青溪', s_pos)
    elif '翠' in s_pos:
        n_pos = correct_dormitory('翠柏', s_pos)
    elif '白' in s_pos:
        if '启真' not in s_pos:
            n_pos = correct_dormitory('白沙', s_pos)
        else:
            n_pos = s_pos
            
    # 食堂
    # 临湖教超除外
    elif '临湖' in s_pos and '教超' not in s_pos:
        n_pos = '临湖食堂'
    elif '麦香' in s_pos:
        n_pos = '麦香餐厅'
    elif '食堂' in s_pos or '休闲' in s_pos or '风味' in s_pos:
        n_pos = '生活区大食堂'
    
    # 教室
    elif '东' in s_pos and '浦东' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '东田径场'
        else:
            n_pos = correct_dormitory('东区教学楼', s_pos)
    elif '西' in s_pos and '银泰' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '西田径场'
        else:
            n_pos = correct_dormitory('西区教学楼', s_pos)
    
    # 安中大楼
    elif '安中' in s_pos:
        if 'A' in s_pos or 'a' in s_pos:
            n_pos = '安中大楼A'
        elif 'B' in s_pos or 'b' in s_pos:
            n_pos = '安中大楼B'
        else:
            n_pos = '安中大楼'
    elif '建工' in s_pos or '大厅' in s_pos:
        n_pos = '建工试验大厅'
    elif '小卖部' in s_pos or '启真便利店' in s_pos:
        n_pos = '白沙启真便利店'
    elif '农' in s_pos:
        n_pos = '农生环'
    else:
        n_pos = s_pos
    new_end_pos.append(n_pos)
    print(s_pos, '-->', n_pos)

安中大楼B --> 安中大楼B
风味食堂 --> 生活区大食堂
翠柏二舍 --> 翠柏2
安中大楼B --> 安中大楼B
西2 --> 西区教学楼2
安中大楼B --> 安中大楼B
翠柏二舍 --> 翠柏2
安中B --> 安中大楼B
风味食堂 --> 生活区大食堂
翠柏二舍 --> 翠柏2
安中B --> 安中大楼B
休闲食堂 --> 生活区大食堂
安中B --> 安中大楼B
东1 --> 东区教学楼1
安中B --> 安中大楼B
翠柏二舍 --> 翠柏2
风味食堂 --> 生活区大食堂
校医院 --> 校医院
安中B --> 安中大楼B
西2 --> 西区教学楼2
风味食堂 --> 生活区大食堂
安中B --> 安中大楼B
翠柏二舍 --> 翠柏2
健身房 --> 健身房
风味食堂 --> 生活区大食堂
翠柏二舍 --> 翠柏2
东6教学楼 --> 东区教学楼
堕落街 --> 堕落街
青溪四舍 --> 青溪4
西2楼食堂 --> 生活区大食堂
青溪四舍 --> 青溪4
西2食堂 --> 生活区大食堂
青溪四舍 --> 青溪4
风味食堂 --> 生活区大食堂
东6教学楼 --> 东区教学楼
堕落街 --> 堕落街
东6教学楼 --> 东区教学楼
青溪四舍 --> 青溪4
南华园 --> 南华园
剑桥公社 --> 剑桥公社
青溪四舍 --> 青溪4
农生环A --> 农生环
白沙3 --> 白沙3
浦东机场 --> 浦东机场
白沙3 --> 白沙3
农A --> 农生环
白沙3 --> 白沙3
农A --> 农生环
白沙3 --> 白沙3
安中大楼 --> 安中大楼
食堂 --> 生活区大食堂
白沙二舍 --> 白沙2
安中大楼 --> 安中大楼
食堂 --> 生活区大食堂
安中大楼 --> 安中大楼
白沙二舍 --> 白沙2
食堂 --> 生活区大食堂
白沙二舍 --> 白沙2
安中 --> 安中大楼
白沙二舍 --> 白沙2
安中 --> 安中大楼
白沙二舍 --> 白沙2
安中大楼 --> 安中大楼
食堂 --> 生活区大食堂
安中 --> 安中大楼
食堂 --> 生活区大食堂
白沙二舍 --> 白沙2
蒙民伟楼 --> 蒙民伟楼
白沙二舍 --> 白沙2
大食堂 --> 生活区大食堂
安中 --> 安中大楼
大食堂 --> 生活区大食堂
安中 --> 安中大

In [312]:
data['修正后到达地点'] = new_end_pos
print(data['修正后到达地点'].unique())

['安中大楼B' '生活区大食堂' '翠柏2' '西区教学楼2' '东区教学楼1' '校医院' '健身房' '东区教学楼' '堕落街' '青溪4'
 '南华园' '剑桥公社' '农生环' '白沙3' '浦东机场' '安中大楼' '白沙2' '蒙民伟楼' '白沙1' '安中大楼A' '临湖食堂'
 '翠柏1' '建工试验大厅' '蓝田4' '东区教学楼4' '西区教学楼1' '紫云3' '图书馆' '小剧场' '格外' '西区教学楼'
 '西区教学楼4' '寝室' '玉泉图书馆' '东区教学楼2' '西田径场' '翠柏4' '码头' '临湖教超' '化学实验中心' '蓝田'
 '东田径场' '紫云' '丹阳' '麦香餐厅' '城西银泰D座' '白沙4' '风雨操场' '文三路' '白沙启真便利店' '白沙'
 '紫金港公交站' '云峰' '机房' '生物实验中心' '小河直街' 'C4' '月牙楼' '网球场' '金工中心' '翠柏']


In [313]:
# 统计出发地点-到达地点
s2e = []
for i, row in data.iterrows():
    start_to_end = row['修正后出发地点'] + '-->' + row['修正后到达地点']
    print(start_to_end)
    s2e.append(start_to_end)

翠柏2-->安中大楼B
安中大楼B-->生活区大食堂
生活区大食堂-->翠柏2
翠柏2-->安中大楼B
安中大楼B-->西区教学楼2
西区教学楼2-->安中大楼B
安中大楼B-->翠柏2
翠柏2-->安中大楼B
安中大楼B-->生活区大食堂
生活区大食堂-->翠柏2
翠柏2-->安中大楼B
安中大楼B-->生活区大食堂
生活区大食堂-->安中大楼B
安中大楼B-->东区教学楼1
东区教学楼1-->安中大楼B
安中大楼B-->翠柏2
翠柏2-->生活区大食堂
生活区大食堂-->校医院
校医院-->安中大楼B
安中大楼B-->西区教学楼2
西区教学楼2-->生活区大食堂
生活区大食堂-->安中大楼B
安中大楼B-->翠柏2
翠柏2-->健身房
健身房-->生活区大食堂
生活区大食堂-->翠柏2
青溪4-->东区教学楼
东区教学楼-->堕落街
堕落街-->青溪4
青溪4-->生活区大食堂
生活区大食堂-->青溪4
青溪4-->生活区大食堂
生活区大食堂-->青溪4
青溪4-->生活区大食堂
生活区大食堂-->东区教学楼
东区教学楼-->堕落街
堕落街-->东区教学楼
东区教学楼-->青溪4
青溪4-->南华园
南华园-->剑桥公社
剑桥公社-->青溪4
白沙3-->农生环
农生环-->白沙3
白沙3-->浦东机场
浦东机场-->白沙3
白沙3-->农生环
农生环-->白沙3
白沙3-->农生环
农生环-->白沙3
白沙2-->安中大楼
安中大楼-->生活区大食堂
生活区大食堂-->白沙2
白沙3-->安中大楼
安中大楼-->生活区大食堂
生活区大食堂-->安中大楼
安中大楼-->白沙2
白沙2-->生活区大食堂
生活区大食堂-->白沙2
白沙2-->安中大楼
安中大楼-->白沙2
白沙2-->安中大楼
安中大楼-->白沙2
白沙2-->安中大楼
安中大楼-->生活区大食堂
生活区大食堂-->安中大楼
安中大楼-->生活区大食堂
生活区大食堂-->白沙2
白沙2-->蒙民伟楼
蒙民伟楼-->白沙2
白沙1-->生活区大食堂
生活区大食堂-->安中大楼
安中大楼-->生活区大食堂
生活区大食堂-->安中大楼
安中大楼-->生活区大食堂
生活区大食堂-->白沙1
白沙1-->生活区大食堂
生活区大食堂-->安中大楼
安中大楼-->生活区大食堂
生活区大食堂-->安中大楼
安中大

In [314]:
new_start_pos = []
for i, row in data.iterrows():
    s_pos = row['出发地点']
    # 宿舍
    # 除 白沙启真便利店，紫金港东操场外， 紫金港公交站外都可修正
    if '蓝' in s_pos:
        n_pos = '蓝田'
    elif '碧' in s_pos:
        n_pos = '碧峰'
    elif '紫云' in s_pos:
        n_pos = '紫云'
    elif '丹' in s_pos:
        n_pos = '丹阳'
    elif '青' in s_pos:
        n_pos = '青溪'
    elif '翠' in s_pos:
        n_pos = '翠柏'
    elif '白' in s_pos:
        if '启真' not in s_pos:
            n_pos = '白沙'
        else:
            n_pos = s_pos
            
    # 食堂
    # 临湖教超除外
    elif '临湖' in s_pos and '教超' not in s_pos:
        n_pos = '临湖食堂'
    elif '麦香' in s_pos:
        n_pos = '麦香餐厅'
    elif '食堂' in s_pos or '休闲' in s_pos or '风味' in s_pos:
        n_pos = '生活区大食堂'
    
    # 教室
    elif '东' in s_pos and '浦东' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '东田径场'
        else:
            n_pos = correct_dormitory('东区教学楼', s_pos)
    elif '西' in s_pos and '银泰' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '西田径场'
        else:
            n_pos = correct_dormitory('西区教学楼', s_pos)
    
    # 安中大楼
    elif '安中' in s_pos:
            n_pos = '安中大楼'
    elif '建工' in s_pos or '大厅' in s_pos:
        n_pos = '建工试验大厅'
    elif '小卖部' in s_pos or '启真便利店' in s_pos:
        n_pos = '白沙启真便利店'
    elif '农' in s_pos:
        n_pos = '农生环'
    else:
        n_pos = s_pos
    new_start_pos.append(n_pos)

In [315]:
new_end_pos = []
for i, row in data.iterrows():
    s_pos = row['到达地点']
    # 宿舍
    # 除 白沙启真便利店，紫金港东操场外， 紫金港公交站外都可修正
    if '蓝' in s_pos:
        n_pos = '蓝田'
    elif '碧' in s_pos:
        n_pos = '碧峰'
    elif '紫云' in s_pos:
        n_pos = '紫云'
    elif '丹' in s_pos:
        n_pos = '丹阳'
    elif '青' in s_pos:
        n_pos = '青溪'
    elif '翠' in s_pos:
        n_pos = '翠柏'
    elif '白' in s_pos:
        if '启真' not in s_pos:
            n_pos = '白沙'
        else:
            n_pos = s_pos
    elif '寝室' in s_pos:
        n_pos = '蓝田'
    # 食堂
    # 临湖教超除外
    elif '临湖' in s_pos and '教超' not in s_pos:
        n_pos = '临湖食堂'
    elif '麦香' in s_pos:
        n_pos = '麦香餐厅'
    elif '食堂' in s_pos or '休闲' in s_pos or '风味' in s_pos:
        n_pos = '生活区大食堂'
    
    # 教室
    elif '东' in s_pos and '浦东' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '东田径场'
        else:
            n_pos = correct_dormitory('东区教学楼', s_pos)
    elif '西' in s_pos and '银泰' not in s_pos and '食堂' not in s_pos:
        if '操' in s_pos:
            n_pos = '西田径场'
        else:
            n_pos = correct_dormitory('西区教学楼', s_pos)
    
    # 安中大楼
    elif '安中' in s_pos:
            n_pos = '安中大楼'
    elif '建工' in s_pos or '大厅' in s_pos:
        n_pos = '建工试验大厅'
    elif '小卖部' in s_pos or '启真便利店' in s_pos:
        n_pos = '白沙启真便利店'
    elif '农' in s_pos:
        n_pos = '农生环'
    else:
        n_pos = s_pos
    new_end_pos.append(n_pos)

In [316]:
print(data['修正后出发地点'].unique())
print('-------------------------------------')
print(data['修正后到达地点'].unique())

['翠柏2' '安中大楼B' '生活区大食堂' '西区教学楼2' '东区教学楼1' '校医院' '健身房' '青溪4' '东区教学楼' '堕落街'
 '南华园' '剑桥公社' '白沙3' '农生环' '浦东机场' '白沙2' '安中大楼' '蒙民伟楼' '白沙1' '翠柏1' '安中大楼A'
 '临湖食堂' '建工试验大厅' '翠柏' '蓝田4' '东区教学楼4' '紫云3' '西区教学楼1' '图书馆' '小剧场' '格外'
 '西区教学楼' '西区教学楼4' '玉泉图书馆' '翠柏4' '董' '西田径场' '码头' '临湖教超' '东区教学楼2' '化学实验中心'
 '蓝田' '东田径场' '紫云' '丹阳' '麦香餐厅' '城西银泰D座' '白沙4' '风雨操场' '文三路' '白沙' '白沙启真便利店'
 '紫金港公交站' '云峰' '机房' '生物实验中心' '小河直街' 'C4' '月牙楼' '网球场' '金工中心']
-------------------------------------
['安中大楼B' '生活区大食堂' '翠柏2' '西区教学楼2' '东区教学楼1' '校医院' '健身房' '东区教学楼' '堕落街' '青溪4'
 '南华园' '剑桥公社' '农生环' '白沙3' '浦东机场' '安中大楼' '白沙2' '蒙民伟楼' '白沙1' '安中大楼A' '临湖食堂'
 '翠柏1' '建工试验大厅' '蓝田4' '东区教学楼4' '西区教学楼1' '紫云3' '图书馆' '小剧场' '格外' '西区教学楼'
 '西区教学楼4' '寝室' '玉泉图书馆' '东区教学楼2' '西田径场' '翠柏4' '码头' '临湖教超' '化学实验中心' '蓝田'
 '东田径场' '紫云' '丹阳' '麦香餐厅' '城西银泰D座' '白沙4' '风雨操场' '文三路' '白沙启真便利店' '白沙'
 '紫金港公交站' '云峰' '机房' '生物实验中心' '小河直街' 'C4' '月牙楼' '网球场' '金工中心' '翠柏']


In [317]:
# 除去浦东机场，C4，董，小河直街，城西银泰D座，格外，码头，玉泉图书馆
start = list(data['修正后出发地点'].unique())
start.remove('浦东机场')
start.remove('城西银泰D座')
start.remove('C4')
start.remove('小河直街')
start.remove('董')
start.remove('格外')
start.remove('码头')
start.remove('玉泉图书馆')
end = list(data['修正后到达地点'].unique())
end.remove('浦东机场')
end.remove('城西银泰D座')
end.remove('C4')
end.remove('小河直街')
end.remove('格外')
end.remove('码头')
end.remove('玉泉图书馆')

In [318]:
s2e = []
for i in range(len(new_start_pos)):
    se = new_start_pos[i] + '-->' + new_end_pos[i]
    s2e.append(se)
data['s2e'] = s2e
print(data['s2e'].unique())

['翠柏-->安中大楼' '安中大楼-->生活区大食堂' '生活区大食堂-->翠柏' '安中大楼-->西区教学楼2' '西区教学楼2-->安中大楼'
 '安中大楼-->翠柏' '生活区大食堂-->安中大楼' '安中大楼-->东区教学楼1' '东区教学楼1-->安中大楼' '翠柏-->生活区大食堂'
 '生活区大食堂-->校医院' '校医院-->安中大楼' '西区教学楼2-->生活区大食堂' '翠柏-->健身房' '健身房-->生活区大食堂'
 '青溪-->东区教学楼' '东区教学楼-->堕落街' '堕落街-->青溪' '青溪-->生活区大食堂' '生活区大食堂-->青溪'
 '生活区大食堂-->东区教学楼' '堕落街-->东区教学楼' '东区教学楼-->青溪' '青溪-->南华园' '南华园-->剑桥公社'
 '剑桥公社-->青溪' '白沙-->农生环' '农生环-->白沙' '白沙-->浦东机场' '浦东机场-->白沙' '白沙-->安中大楼'
 '生活区大食堂-->白沙' '安中大楼-->白沙' '白沙-->生活区大食堂' '白沙-->蒙民伟楼' '蒙民伟楼-->白沙'
 '安中大楼-->临湖食堂' '临湖食堂-->健身房' '健身房-->安中大楼' '西区教学楼2-->翠柏' '生活区大食堂-->建工试验大厅'
 '建工试验大厅-->校医院' '翠柏-->建工试验大厅' '建工试验大厅-->安中大楼' '蓝田-->西区教学楼2' '蓝田-->生活区大食堂'
 '建工试验大厅-->蓝田' '青溪-->东区教学楼4' '东区教学楼4-->青溪' '青溪-->校医院' '校医院-->堕落街'
 '校医院-->青溪' '南华园-->青溪' '青溪-->剑桥公社' '紫云-->西区教学楼1' '西区教学楼1-->紫云'
 '西区教学楼1-->生活区大食堂' '生活区大食堂-->紫云' '翠柏-->西区教学楼2' '西区教学楼2-->图书馆'
 '图书馆-->生活区大食堂' '西区教学楼2-->小剧场' '小剧场-->翠柏' '翠柏-->翠柏' '生活区大食堂-->格外'
 '格外-->翠柏' '白沙-->校医院' '校医院-->西区教学楼2' '西区教学楼2-->临湖食堂' '临湖食堂-->西区教学楼1'
 '西区教学楼1-->白沙' '白沙-->西区教学楼' '西区教

In [319]:
data = data[data['修正后出发地点'].isin(start) & data['修正后到达地点'].isin(end)]

In [1]:
import requests
import time
key = 'dd6a138c5da8b9a90c80eddbc42662ea'

In [12]:
def get_gps(pos):
    pos = '浙江大学紫金港校区' + pos
    url = 'http://restapi.amap.com/v3/place/text?keywords=' + pos + '&city=hangzhou&output=xml&offset=1&page=1&key=dd6a138c5da8b9a90c80eddbc42662ea&extensions=all'
    headers = {'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}
    html = requests.get(url, headers=headers)
    gps = html.text.split('<location>')[1].split('</location>')[0]
    return gps

In [ ]:
start_gps = []
end_gps = []
for i, row in data.iterrows():
    s = row['修正后出发地点']
    e = row['修正后到达地点']
    try:
        s_gps = get_gps(s)
        e_gps = get_gps(e)
    except:
        s_gps = '0,0'
        e_gps = '0,0'
    print('出发地点', s, ':',s_gps, '-----', '到达地点', e, ':', e_gps)
    start_gps.append(s_gps)
    end_gps.append(e_gps)
    time.sleep(0.1)

In [7]:
data = pd.read_csv('data.csv').iloc[:, 1:]
data.head()

,用户ID,星期,日期,Trip_ID,出行目的,第几次出行,出发地点,到达地点,出发时间,达到时间,...,自行车保有,电动车保有,汽车保有,校内主要出行方式,校外主要出行方式,修正后出发地点,修正后到达地点,s2e,start_gps,end_gps
0,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0001,4,1,翠柏二舍,安中大楼B,7:40,8:03,...,2,2,2,1-2,1-2-7-8-9,翠柏2,安中大楼B,翠柏-->安中大楼,"120.086135,30.309519","120.083456,30.301339"
1,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0002,3,2,安中大楼B,风味食堂,11:15,11:25,...,2,2,2,1-2,1-2-7-8-9,安中大楼B,生活区大食堂,安中大楼-->生活区大食堂,"120.083456,30.301339","120.125184,30.267103"
2,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0003,6,3,风味食堂,翠柏二舍,11:40,11:46,...,2,2,2,1-2,1-2-7-8-9,生活区大食堂,翠柏2,生活区大食堂-->翠柏,"120.125184,30.267103","120.086135,30.309519"
3,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0004,4,4,翠柏二舍,安中大楼B,14:15,14:31,...,2,2,2,1-2,1-2-7-8-9,翠柏2,安中大楼B,翠柏-->安中大楼,"120.086135,30.309519","120.083456,30.301339"
4,04D0BF86-23A4-498C-8DAD-1984B238772C,Thu,2018/04/17,04D0BF86-23A4-498C-8DAD-1984B238772C0005,2,5,安中大楼B,西2,14:37,14:42,...,2,2,2,1-2,1-2-7-8-9,安中大楼B,西区教学楼2,安中大楼-->西区教学楼2,"120.083456,30.301339","120.084998,30.301415"


In [16]:
start_gps = []
end_gps = []
for i, row in data.iterrows():
    if row['start_gps'] == '0,0' or row['end_gps'] =='0,0':
        data.loc[i, ['start_gps']] = get_gps(row['修正后出发地点'])
        data.loc[i, ['end_gps']] = get_gps(row['修正后到达地点'])

In [17]:
for i, row in data.iterrows():
    if row['start_gps'] == '0,0' or row['end_gps'] =='0,0':
        print(row['s2e'])

In [18]:
data.to_csv('data.csv')

# 添加距离

In [25]:
url = 'http://restapi.amap.com/v3/direction/walking?origin={}&destination={}&key={}'

In [28]:
start_gps = data.loc[10, 'start_gps']
end_gps = data.loc[10, 'end_gps']
path = url.format(start_gps, end_gps, key)
print(path)

http://restapi.amap.com/v3/direction/walking?origin=120.086135,30.309519&destination=120.083456,30.301339&key=dd6a138c5da8b9a90c80eddbc42662ea


In [29]:
headers = {'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}
html = requests.get(path, headers=headers)
print(html.text)

{"status":"1","info":"ok","infocode":"10000","count":"1","route":{"origin":"120.086135,30.309519","destination":"120.083456,30.301339","paths":[{"distance":"1172","duration":"938","steps":[{"instruction":"步行61米左转","orientation":[],"road":[],"distance":"61","duration":"49","polyline":"120.086113,30.309614;120.085495,30.30953","action":"左转","assistant_action":[],"walk_type":"0"},{"instruction":"向南步行389米右转","orientation":"南","road":[],"distance":"389","duration":"311","polyline":"120.085487,30.309526;120.085556,30.309149;120.085556,30.309149;120.085617,30.308849;120.085617,30.308849;120.085815,30.307896;120.085815,30.307896;120.085869,30.307623;120.085869,30.307573;120.085869,30.307573;120.086159,30.306061","action":"右转","assistant_action":[],"walk_type":"0"},{"instruction":"向西南步行262米右转","orientation":"西南","road":[],"distance":"262","duration":"210","polyline":"120.086159,30.306053;120.085434,30.305746;120.085197,30.305573;120.085075,30.305473;120.084862,30.305195;120.084747,30.304945;120

In [32]:
import json
text = json.loads(html.text)

In [37]:
text['route']['paths'][0]['distance']

'1172'

In [42]:
def get_distance(start_gps, end_gps):
    key = 'dd6a138c5da8b9a90c80eddbc42662ea'
    url = 'http://restapi.amap.com/v3/direction/walking?origin={}&destination={}&key={}'.format(start_gps, end_gps, key)
    headers = {'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"}
    html = requests.get(url, headers=headers)
    text = json.loads(html.text)
    distance = text['route']['paths'][0]['distance']
    return distance

In [43]:
distances = []
for i, row in data.iterrows():
    start_gps = row['start_gps']
    end_gps = row['end_gps']
    try:
        distance = get_distance(start_gps, end_gps)
    except:
        time.sleep(0.5)
        try:
            distance = get_distance(start_gps, end_gps)
        except:
            distance = -1
    distances.append(distance)
    print(start_gps, ',', end_gps, ':', distance)
    time.sleep(0.05)

120.086135,30.309519 , 120.083456,30.301339 : 1172
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.086135,30.309519 : 8066
120.086135,30.309519 , 120.083456,30.301339 : 1172
120.083456,30.301339 , 120.084998,30.301415 : 160
120.084998,30.301415 , 120.083456,30.301339 : 160
120.083456,30.301339 , 120.086135,30.309519 : 1172
120.086135,30.309519 , 120.083456,30.301339 : 1172
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.086135,30.309519 : 8066
120.086135,30.309519 , 120.083456,30.301339 : 1172
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.083456,30.301339 : 7147
120.083456,30.301339 , 120.090673,30.302388 : 1111
120.090673,30.302388 , 120.083456,30.301339 : 1111
120.083456,30.301339 , 120.086135,30.309519 : 1172
120.086135,30.309519 , 120.125184,30.267103 : 8066
120.125184,30.267103 , 120.088450,30.309169 : 7589
120.088450,30.309169 , 120.083456,30.301339 : 1281
120.083456,30.301339 , 120.084998

120.091614,30.297916 , 120.086379,30.308393 : 1632
120.086379,30.308393 , 120.125184,30.267103 : 7891
120.125184,30.267103 , 120.084998,30.301415 : 6967
120.084998,30.301415 , 120.084109,30.307723 : 705
120.084109,30.307723 , 120.084998,30.301415 : 705
120.084998,30.301415 , 120.125184,30.267103 : 6967
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.091614,30.297916 : 1632
120.091614,30.297916 , 120.086379,30.308393 : 1632
120.086379,30.308393 , 120.125184,30.267103 : 7891
120.125184,30.267103 , 120.084998,30.301415 : 6967
120.084998,30.301415 , 120.125184,30.267103 : 6967
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.084998,30.301415 : 914
120.084998,30.301415 , 120.084998,30.301415 : 1
120.084998,30.301415 , 120.125184,30.267103 : 6967
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.091614,30.297916 : 1632
120.091614,30.297916 , 120.086379,30.308393 : 1632
120.086379,30.308393 , 120.084998,30.

120.089053,30.309222 , 120.088568,30.303810 : 783
120.088568,30.303810 , 120.089053,30.309222 : 783
120.089053,30.309222 , 120.088568,30.303810 : 783
120.086379,30.308393 , 120.089053,30.309222 : 320
120.089053,30.309222 , 120.086379,30.308393 : 320
120.086379,30.308393 , 120.125184,30.267103 : 7891
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.125184,30.267103 : 7891
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.125184,30.267103 : 7891
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.125184,30.267103 : 7891
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.083456,30.301339 : 1089
120.083456,30.301339 , 120.086379,30.308393 : 1089
120.086379,30.308393 , 120.086952,30.308580 : 9
120.086952,30.308580 , 120.086379,30.308393 : 9
120.086379,30.308393 , 120.125184,30.267103 : 7891
120.125184,30.267103 , 120.086379,30.308393 : 7891
120.086379,30.308393 , 120.086379,30.30839

120.087411,30.308813 , 120.083456,30.301339 : 1173
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.083456,30.301339 : 7147
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.083456,30.301339 : 7147
120.083456,30.301339 , 120.087411,30.308813 : 1173
120.087411,30.308813 , 120.083456,30.301339 : 1173
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.083456,30.301339 : 7147
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.083456,30.301339 : 7147
120.083456,30.301339 , 120.087411,30.308813 : 1173
120.087411,30.308813 , 120.083456,30.301339 : 1173
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.083456,30.301339 : 7147
120.083456,30.301339 , 120.091614,30.297916 : 1252
120.091614,30.297916 , 120.083456,30.301339 : 1252
120.083456,30.301339 , 120.125184,30.267103 : 7147
120.125184,30.267103 , 120.083456,30.301339 : 7147
120.083456,30.301339 , 120.0874

In [44]:
data['distance'] = distances

In [45]:
for i, row in data.iterrows():
    print(row['s2e'], ':', row['distance'])

翠柏-->安中大楼 : 1172
安中大楼-->生活区大食堂 : 7147
生活区大食堂-->翠柏 : 8066
翠柏-->安中大楼 : 1172
安中大楼-->西区教学楼2 : 160
西区教学楼2-->安中大楼 : 160
安中大楼-->翠柏 : 1172
翠柏-->安中大楼 : 1172
安中大楼-->生活区大食堂 : 7147
生活区大食堂-->翠柏 : 8066
翠柏-->安中大楼 : 1172
安中大楼-->生活区大食堂 : 7147
生活区大食堂-->安中大楼 : 7147
安中大楼-->东区教学楼1 : 1111
东区教学楼1-->安中大楼 : 1111
安中大楼-->翠柏 : 1172
翠柏-->生活区大食堂 : 8066
生活区大食堂-->校医院 : 7589
校医院-->安中大楼 : 1281
安中大楼-->西区教学楼2 : 160
西区教学楼2-->生活区大食堂 : 6967
生活区大食堂-->安中大楼 : 7147
安中大楼-->翠柏 : 1172
翠柏-->健身房 : 816
健身房-->生活区大食堂 : 7365
生活区大食堂-->翠柏 : 8066
青溪-->东区教学楼 : 2009
东区教学楼-->堕落街 : 1491
堕落街-->青溪 : 697
青溪-->生活区大食堂 : 8142
生活区大食堂-->青溪 : 8142
青溪-->生活区大食堂 : 8142
生活区大食堂-->青溪 : 8142
青溪-->生活区大食堂 : 8142
生活区大食堂-->东区教学楼 : 6091
东区教学楼-->堕落街 : 1491
堕落街-->东区教学楼 : 1491
东区教学楼-->青溪 : 2009
青溪-->南华园 : 1401
南华园-->剑桥公社 : 1577
剑桥公社-->青溪 : 1792
白沙-->农生环 : 1522
农生环-->白沙 : 1522
白沙-->农生环 : 1522
农生环-->白沙 : 1522
白沙-->农生环 : 1522
农生环-->白沙 : 1522
白沙-->安中大楼 : 1089
安中大楼-->生活区大食堂 : 7147
生活区大食堂-->白沙 : 7891
白沙-->安中大楼 : 1173
安中大楼-->生活区大食堂 : 7147
生活区大食堂-->安中大楼 : 7147
安中大楼-->白沙 : 10

西区教学楼1-->西田径场 : 481
西田径场-->堕落街 : 829
堕落街-->蓝田 : 935
蓝田-->生活区大食堂 : 8220
生活区大食堂-->东区教学楼1 : 6652
东区教学楼1-->堕落街 : 802
堕落街-->蓝田 : 935
蓝田-->东区教学楼1 : 1546
东区教学楼1-->西田径场 : 873
西田径场-->蓝田 : 604
翠柏-->安中大楼 : 274
安中大楼-->生活区大食堂 : 7147
安中大楼-->健身房 : 356
健身房-->安中大楼 : 356
安中大楼-->翠柏 : 274
翠柏-->安中大楼 : 274
安中大楼-->西区教学楼1 : 160
西区教学楼1-->生活区大食堂 : 6967
生活区大食堂-->安中大楼 : 7147
安中大楼-->翠柏 : 1106
翠柏-->安中大楼 : 274
安中大楼-->生活区大食堂 : 7147
生活区大食堂-->安中大楼 : 7147
安中大楼-->健身房 : 356
健身房-->安中大楼 : 356
安中大楼-->翠柏 : 274


In [47]:
# 修正大食堂位置
for i, row in data.iterrows():
    if row['修正后出发地点'] == '生活区大食堂':
        data.loc[i, ['start_gps']] = get_gps('大食堂')
    if row['修正后到达地点'] == '生活区大食堂':
        data.loc[i, ['end_gps']] = get_gps('大食堂')

In [49]:
for i, row in data.iterrows():
    if row['修正后出发地点'] == '生活区大食堂' or row['修正后到达地点'] == '生活区大食堂':
        distance = get_distance(row['start_gps'], row['end_gps'])
        print(row['s2e'], ':', distance)
        data.loc[i, ['distance']] = distance

安中大楼-->生活区大食堂 : 851
生活区大食堂-->翠柏 : 263
安中大楼-->生活区大食堂 : 851
生活区大食堂-->翠柏 : 263
安中大楼-->生活区大食堂 : 851
生活区大食堂-->安中大楼 : 851
翠柏-->生活区大食堂 : 263
生活区大食堂-->校医院 : 372
西区教学楼2-->生活区大食堂 : 752
生活区大食堂-->安中大楼 : 851
健身房-->生活区大食堂 : 495
生活区大食堂-->翠柏 : 263
青溪-->生活区大食堂 : 172
生活区大食堂-->青溪 : 172
青溪-->生活区大食堂 : 172
生活区大食堂-->青溪 : 172
青溪-->生活区大食堂 : 172
生活区大食堂-->东区教学楼 : 1705
安中大楼-->生活区大食堂 : 851
生活区大食堂-->白沙 : 278
安中大楼-->生活区大食堂 : 851
生活区大食堂-->安中大楼 : 851
白沙-->生活区大食堂 : 278
生活区大食堂-->白沙 : 278
安中大楼-->生活区大食堂 : 851
生活区大食堂-->安中大楼 : 851
安中大楼-->生活区大食堂 : 851
生活区大食堂-->白沙 : 278
白沙-->生活区大食堂 : 248
生活区大食堂-->安中大楼 : 851
安中大楼-->生活区大食堂 : 851
生活区大食堂-->安中大楼 : 851
安中大楼-->生活区大食堂 : 851
生活区大食堂-->白沙 : 248
白沙-->生活区大食堂 : 248
生活区大食堂-->安中大楼 : 851
安中大楼-->生活区大食堂 : 851
生活区大食堂-->安中大楼 : 851
安中大楼-->生活区大食堂 : 851
生活区大食堂-->安中大楼 : 851
白沙-->生活区大食堂 : 248
生活区大食堂-->安中大楼 : 851
安中大楼-->生活区大食堂 : 851
生活区大食堂-->安中大楼 : 851
翠柏-->生活区大食堂 : 823
生活区大食堂-->安中大楼 : 851
翠柏-->生活区大食堂 : 823
生活区大食堂-->安中大楼 : 851
翠柏-->生活区大食堂 : 823
生活区大食堂-->安中大楼 : 851
生活区大食堂-->建工试验大厅 : 840
生活区大食堂-->建工试验大厅 

In [50]:
data.to_csv('data.csv')